In [1]:
import re
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [20]:
df = pd.read_csv("./IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [50]:
labels = df['sentiment'].apply(lambda x:int(x == 'positive')).values

In [25]:
SYMBOL_FILTER = re.compile("[!,.\"\':?()]")
SPECIAL_CHAR_FILTER = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

df['review'] = df.review.apply(lambda x:SPECIAL_CHAR_FILTER.sub("",SYMBOL_FILTER.sub("",x)).lower())
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically theres a family where a little boy j...,negative
4,petter matteis love in the time of money is a ...,positive


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LogisticRegression

### Using logistic regression with 1 fold train test split.

In [35]:
cvt = CountVectorizer()
cvt.fit(features['review'].values)

vects = cvt.transform(features['review'].values)

In [36]:
X,x,Y,y = train_test_split(vects,labels,random_state = 42)

In [37]:
log = LogisticRegression(C=0.3,n_jobs=4,solver="saga")
log.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=0.3, n_jobs=4, solver='saga')

In [38]:
log.score(x,y)

0.88512

In [39]:
sent = "I liked movie"
log.predict(cvt.transform([sent]))

array(['positive'], dtype=object)

In [40]:
sent = "I didnt like the movie"
log.predict(cvt.transform([sent]))

array(['negative'], dtype=object)

### Using KFolds

In [53]:
folds = KFold(n_splits=5,shuffle=True)

In [ ]:
for train,test in folds.split(vects,labels):
    log = LogisticRegression(C=0.3,n_jobs=-1,solver="saga",max_iter=150)
    log.fit(vects[train],labels[train])
    
    print (f'Fold Score : {log.score(vects[test],labels[test])}')